# Query Approaches Experiment
Experiment to compare the performance of several different ways to query data and compute statistics from populations of forecast and observed data pairs from parquet files.  This includes duckdb, pandas and dask dataframes, as well as a hybrid approach.

In [ ]:
%matplotlib inline

# adding project dirs to path so code may be referenced from the notebook
import sys
sys.path.insert(0, '../../evaluation')
sys.path.insert(0, '../../evaluation/queries')

# DuckDB
This approach is a straight DuckDB approach where timeseries are queried and metrics calculated in the SQL query.

In [ ]:
%%capture
!pip install duckdb

In [ ]:
import config
import queries
import duckdb

In [ ]:
%%time
query = queries.calculate_catchment_metrics(
    config.MEDIUM_RANGE_FORCING_PARQUET,
    config.FORCING_ANALYSIS_ASSIM_PARQUET,
    group_by=["catchment_id"],
    order_by=["observed_average"],
    filters=[
        {
            "column": "catchment_id",
            "operator": "like",
            "value": "18%"
        },
        {
            "column": "reference_time",
            "operator": "=",
            "value": "2022-12-25 00:00:00"
        },
    ]
)
df = duckdb.query(query).to_df()
df

# Pandas
Using this approach we open the parquet files using pandas and calculate the metrics using pandas groupby and aggregate functionality. We only caculate two simple metrics because even with that the performance was not too good.  More metrics would only make it worse.

In [ ]:
import pandas as pd

In [ ]:
%%time
# load forecast data
df_forecast = pd.read_parquet(config.MEDIUM_RANGE_FORCING_PARQUET)
df_forecast = df_forecast[
        (df_forecast["catchment_id"].str.startswith("18")) & 
        (df_forecast["reference_time"] == pd.Timestamp(2022,12,25,0,0,0))
     ]

# load obersved data
df_observed = pd.read_parquet(config.FORCING_ANALYSIS_ASSIM_PARQUET)
df_observed = df_observed[
        (df_observed["catchment_id"].str.startswith("18"))
     ]

# join forecast and observed
df_joined = pd.merge(
    df_forecast,
    df_observed,
    on=["catchment_id","value_time"],
    suffixes=["_forecast","_observed"],
    how="inner"
)[["catchment_id","reference_time","value_time","value_forecast", "value_observed"]]

# groupby and aggregate
df_joined.groupby("catchment_id")[["value_forecast","value_observed"]].agg(
        average_forecast = ("value_forecast", "mean"),
        average_observed = ("value_observed", "mean")
    )

# Dask
This approach is very similar to the Pandas approach but uses a dask dataframe.  Performance is slightly better, but not much.

In [ ]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)
cluster

In [ ]:
import dask.dataframe as dd

In [ ]:
%%time
# load forecast data
ddf_forecast = dd.read_parquet(config.MEDIUM_RANGE_FORCING_PARQUET)
ddf_forecast = ddf_forecast[
        (ddf_forecast["catchment_id"].str.startswith("18")) & 
        (ddf_forecast["reference_time"] == pd.Timestamp(2022,12,25,0,0,0))
     ]

# load obersved data
ddf_observed = dd.read_parquet(config.FORCING_ANALYSIS_ASSIM_PARQUET)
ddf_observed = ddf_observed[
        (ddf_observed["catchment_id"].str.startswith("18"))
     ]

# join forecast and observed
ddf_joined = dd.merge(
    ddf_forecast,
    ddf_observed,
    on=["catchment_id","value_time"],
    suffixes=["_forecast","_observed"],
    how="inner"
)

# groupby and aggregate
ddf_joined.groupby("catchment_id")[["value_forecast","value_observed"]].agg(
        average_forecast = ("value_forecast", "mean"),
        average_observed = ("value_observed", "mean")
    ).compute()

# Hybrid
The hyrid approach uses DuckDB to query out timeseries pairs and then uses pandas to calculate some statistics.  This approach is likely good for smaller datasets, such as forecasts at a single location where you want to calculate non-standard metrics that are difficult to calculate 

In [ ]:
%%time
query = queries.get_joined_catchment_timeseries(
    config.MEDIUM_RANGE_FORCING_PARQUET,
    config.FORCING_ANALYSIS_ASSIM_PARQUET,
    filters=[
        {
            "column": "catchment_id",
            "operator": "==",
            "value": "1801010101"
        }
    ]
)
df = duckdb.query(query).to_df()
df.groupby(["catchment_id","lead_time"])[["forecast_value","observed_value"]].agg(
        average_forecast = ("forecast_value", "mean"),
        average_observed = ("observed_value", "mean")
    )

In [ ]:
%%time
query = queries.calculate_catchment_metrics(
    config.MEDIUM_RANGE_FORCING_PARQUET,
    config.FORCING_ANALYSIS_ASSIM_PARQUET,
    group_by=["catchment_id","lead_time"],
    order_by=["catchment_id","lead_time"],
    filters=[
         {
            "column": "catchment_id",
            "operator": "==",
            "value": "1801010101"
        }
    ]
)
df = duckdb.query(query).to_df()
df[["catchment_id","lead_time","forecast_average","observed_average"]]

# Conclusion
DuckDB seems to be the fastest way to query and compute metrics and statistics accross large populations of data. For smaller datasets, say just a few locations, pulling the timeseries out and working in Pandas can work and has the benefit of having the power to Pandas to resample, slica e and dice the data in ways that may be difficult in DuckDB.